![](https://github.com/ultralytics/docs/releases/download/0/object-detection-examples.avif)

### 1️⃣ YOLO Object Detection on Custom Images

```python
from ultralytics import YOLO
from PIL import Image
import numpy as np

# Load pre-trained YOLOv8 model
model = YOLO('yolov8s.pt')

# Load an image
image_path = 'path/to/your/image.jpg'
image = Image.open(image_path).convert('RGB')
image_np = np.array(image)

# Run detection
results = model.predict(image_np, conf=0.25)  # adjust confidence
res = results[0]

# Display annotated image
annotated = res.plot()
Image.fromarray(annotated)

# Access bounding boxes
for box, cls, conf in zip(res.boxes.xyxy, res.boxes.cls, res.boxes.conf):
    print(f"Class: {model.names[int(cls)]}, Confidence: {conf:.2f}, Box: {box.numpy()}")
```

---



In [3]:
#!pip install ultralytics pillow

In [6]:
from ultralytics import YOLO
from PIL import Image
import numpy as np
import requests
from io import BytesIO

# Load pre-trained YOLOv8 model
model = YOLO('yolov8s.pt')

# Load an image from HTTPS URL
image_url = 'https://ultralytics.com/images/zidane.jpg'
response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert('RGB')
image_np = np.array(image)

# Run detection
results = model.predict(image_np, conf=0.25)  # adjust confidence threshold
res = results[0]

# Display annotated image
annotated = res.plot()
Image.fromarray(annotated)

# Access bounding boxes
for box, cls, conf in zip(res.boxes.xyxy, res.boxes.cls, res.boxes.conf):
    print(f"Class: {model.names[int(cls)]}, Confidence: {conf:.2f}, Box: {box.numpy()}")



0: 384x640 2 persons, 1 tie, 73.0ms
Speed: 11.2ms preprocess, 73.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Class: person, Confidence: 0.89, Box: [     139.35      201.37      1095.8      712.25]
Class: person, Confidence: 0.88, Box: [     746.92      41.954      1134.1      713.03]
Class: tie, Confidence: 0.60, Box: [     436.41      434.01      534.15      716.92]


![](https://github.com/ultralytics/docs/releases/download/0/image-classification-examples.avif)

### 2️⃣ YOLO Classification

```python
from ultralytics import YOLO

# Load YOLO classification model
model = YOLO('yolov8n-cls.pt')

# Train on classification dataset
# dataset YAML example: path to train/images, val/images, and classes
model.train(data='path/to/classification.yaml', epochs=30, imgsz=224)

# Predict on a single image
results = model.predict('path/to/test/image.jpg')
print(results)
results.show()  # displays predicted class
```

---



### **What `model.train()` does**

When you call:

```python
model.train(data='path/to/classification.yaml', epochs=30, imgsz=224)
```

you are telling YOLOv8 to **fine-tune the pre-trained model** on your own dataset. Here’s what happens step by step:

1. **Load the dataset**

   * `data='path/to/classification.yaml'` points to a YAML file describing your dataset:

     ```yaml
     train: path/to/train/images
     val: path/to/val/images
     nc: 5  # number of classes
     names: ['cat', 'dog', 'horse', 'car', 'person']
     ```
   * YOLOv8 reads all images and labels from the `train` and `val` directories.

2. **Set up the model for training**

   * `yolov8n-cls.pt` is a **classification model**, not detection.
   * The model uses your data to adjust weights for predicting **class labels** for each input image.

3. **Training loop**

   * Runs for `epochs=30` iterations over the dataset.
   * Each batch of images goes through **forward pass → loss computation → backpropagation → weight update**.
   * Optimizer, learning rate schedule, and data augmentation are handled automatically by Ultralytics.

4. **Image resizing**

   * `imgsz=224` resizes each image to 224×224 pixels for training (common for classification tasks).

5. **Validation**

   * After each epoch, the model runs on the validation set and reports metrics like accuracy, loss, F1-score, etc.

6. **Checkpoint saving**

   * YOLO automatically saves weights in `runs/classify/train/weights/` (by default).
   * You can load the best checkpoint later with `YOLO('runs/classify/train/weights/best.pt')`.

---

### **In short**

`model.train()` is the high-level command that:

* Prepares the dataset
* Fine-tunes the pre-trained YOLO model on your data
* Tracks metrics
* Saves checkpoints

You don’t need to manually write PyTorch training loops — Ultralytics handles it for you.


![](https://github.com/ultralytics/docs/releases/download/0/ultralytics-solutions-thumbnail.avif)

### 3️⃣ YOLO for Regression / Custom Numeric Labels

```python
from ultralytics import YOLO

# Load YOLO model
model = YOLO('yolov8s.pt')

# Prepare a dataset where labels are continuous values
# Example: keypoints, bounding box coordinates, or custom regression values

model.train(data='path/to/custom_regression.yaml', epochs=50, imgsz=640)

# Predict on new image
results = model.predict('path/to/test/image.jpg')
print(results)
```

---




### **Why we didn’t use a separate model for regression**

1. **YOLOv8 “detection” models are inherently regression-capable**

   * YOLO’s core task is **predicting bounding boxes**, which are continuous values (x1, y1, x2, y2).
   * So the base detection model (`yolov8s.pt`) already contains the architecture for regression.

2. **Detection vs. regression**

   * Detection = regression for bounding boxes + classification for object labels.
   * Custom regression tasks (e.g., predicting keypoints, angles, object coordinates, or continuous numeric labels) are **structurally similar to bounding box regression**.
   * That means we can reuse a detection model like `yolov8s.pt` and just change the dataset and loss to output your continuous values.

3. **Why not a special “regression model”?**

   * Ultralytics doesn’t ship a separate YOLO model for generic regression tasks.
   * Instead, you can **reuse the detection backbone** and train it on any continuous labels in a `data.yaml`.
   * You just define the label format in your dataset and the model’s loss function handles it automatically.

4. **What changes for regression?**

   * Dataset YAML points to images and **continuous label files**.
   * You may customize `nc` (number of regression outputs per object).
   * `model.train()` works the same way as detection/classification — it just optimizes a regression loss instead of classification loss.

---

### **In short**

* For classification → use `yolov8n-cls.pt` (classification model).
* For detection or regression → use `yolov8s.pt` (detection model), because it already predicts continuous outputs.
* The model type depends on the **output structure**, not the task name.




### Notes & References

* **YOLOv8 Docs:** [https://docs.ultralytics.com](https://docs.ultralytics.com)
* **Detection Tutorial:** [https://docs.ultralytics.com/tasks/detect/](https://docs.ultralytics.com/tasks/detect/)
* **Classification Tutorial:** [https://docs.ultralytics.com/tasks/classify/](https://docs.ultralytics.com/tasks/classify/)